# Dataset Erupciones volcánicas

## Objetivo:

Trabajando con el dataset descargado del siguiente [link](https://public.opendatasoft.com/explore/dataset/significant-volcanic-eruption-database/table/), armar un nuevo archivo modificando los datos y reorganizando las columnas

Primero importo los módulos con los que voy a trabajar:
- El módulo os para definir las rutas de los archivos
- El múdulo csv para trabajar con el archivo de origen de datos y crear el nuevo
- El módulo json para trabajar con el archivo de traducción de palabaras
- El módulo sys para cortar la ejecución en caso de error

In [1]:
import os
import csv
import json
import sys


### 1. Obtención de datos de los archivos

Uilizo una función para obetener los datos del dataset original 
- La ruta para acceder al archivo la armo de acuerdo a la posición en la que se encuentra el archivo en el que estoy trabajando.
- Ultilizo with para abrir el archivo y asegurarme que se cierre una vez finalizado el proceso.
- Dado que es un archivo csv itero con el objeto reader del módulo csv para devolver:
 - Una variable con los nombres de las columnas (primera fila del archivo). Es una lista de strings.
 - Una variable con el resto de los datos. Es una lista de listas de strings, cada lista interna se corresponde a una fila.
- En caso de que no se encuentre el archivo original el controlador de excepciones termina la ejecución con un mensaje de error.

In [2]:
def contenido_archivo_csv():
    """Devolver la cabecera y el contenido de un archivo de tipo csv."""
    nombre_archivo = "significant-volcanic-eruption-database.csv"
    ruta_particular = ""
    ruta_archivo_erupciones = os.path.join(os.getcwd(), ruta_particular, nombre_archivo)
    try:
        with open(ruta_archivo_erupciones, "r", newline="", encoding="UTF-8") as archivo_erupciones:
            csv_reader = csv.reader(archivo_erupciones, delimiter=";")
            cabecera = csv_reader.__next__()
            contenido = [line for line in csv_reader]
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{nombre_archivo}'")
        sys.exit(1)
    else:
        return cabecera, contenido

cabecera, erupciones_original = contenido_archivo_csv()


En otra función obtengo los datos del archivo de traducciones:
- Con la función load del módulo json cargo a una variable los datos del archivo json que contiene las traducciones para el tipo de volcán. Es un diccionario que contiene como pares clave: valor cada tipo y su traducción.
- Al igual que antes armo la ruta a partir de la posición del archivo actual, uso with para abrir y cerrar el archivo y doy un mensaje de error si no se encuentra el archivo.


In [3]:
def contenido_archivo_json():
    """Devolver el contenido de un archivo de tipo json."""
    nombre_archivo = "tipo_volcan_ENG_ESP.json"
    ruta_particular = ""
    ruta_archivo_traduccion = os.path.join(os.getcwd(), ruta_particular, nombre_archivo)
    try:
        with open(ruta_archivo_traduccion, "r", newline="", encoding="UTF-8") as archivo_traduccion:
            contenido = json.load(archivo_traduccion)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{nombre_archivo}'")
        sys.exit(1)
    else:
        return contenido

traducciones = contenido_archivo_json()


### 2. Procesamiento de los datos

Realizo los cambios requeridos a los datos:
- _Traducir el tipo de volcán al español_: la función recibe el tipo de volcán y lo utiliza como clave para acceder al diccionario de traducciones, obteniendo como valor el tipo en español.
- _Convertir “Flag Tsunami” y “Flag Earthquake” a booleanos_: la función recibe el string de la columna y devuelve False si es una cadena vacía y True si tiene valor.
- _Transformar la columna “Name” en una nueva columna “Name and country” para que tenga una combinación del nombre del volcán con el país entre paréntesis_: la función recibe el nombre y el país y devuelve una sola variable con el formato deseado.


In [4]:
def traducir(cadena, traducciones):
    """Devolver la traducción de la palabra de acuerdo al diccionario recibido como parámtro. El diccionario tiene
    como pares clave-valor la palabra y su traducción."""
    traduccion = traducciones[cadena]
    return traduccion


In [5]:
def convertir_a_booleano(dato):
    """Convertir el dato de texto a booleano de acuerdo a la presencia o ausencia de valor."""
    bandera_a_booleano = True if dato != "" else False
    return bandera_a_booleano


In [6]:
def nombre_pais(nombre, pais):
    """Unir el nombre del volcán con el país entre paréntesis."""
    return nombre + " (" + pais + ")"


Iterando en un list comprehension sobre la variable con los datos originales, aplico los cambios a los datos y me quedo con las columnas pedidas en el orden especificado (_“Year”, “Volcanic Explosivity Index”, “Volcano Type”, “Flag Tsunami”, “Flag Earthquake” y “Name and country”_)

In [7]:
def procesar_datos(datos):
    erupciones_procesado = [
        [linea[0], linea[11], traducir(linea[9], traducciones),
         convertir_a_booleano(linea[3]), convertir_a_booleano(linea[4]), nombre_pais(linea[5], linea[7])]
        for linea in datos
        ]
    return erupciones_procesado

erupciones_procesado = procesar_datos(erupciones_original)


### 3. Armado del dataset con las modificaciones

Para finalizar genero el archivo csv a partir de la variable con las modificaciones.
- Armo la ruta a partir de la posición del archivo actual y uso with para abrir y cerrar el archivo.
- Creo el archivo utilizando el modo escritura e itero con el objeto writer de csv para escribir primero la cabecera y luego cada fila de datos.

In [8]:
nombre_archivo = "dataset_volcanes.csv"
ruta_particular = ".."
ruta_archivo_modificaciones = os.path.join(os.getcwd(), ruta_particular, nombre_archivo)
with open(ruta_archivo_modificaciones, "w", newline="", encoding="UTF-8") as archivo_modificaciones:
    writer = csv.writer(archivo_modificaciones)
    writer.writerow([cabecera[0], cabecera[11], cabecera[9],
                     cabecera[3], cabecera[4], "Name and country"
                     ])
    writer.writerows(erupciones_procesado)
